<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/code/descriptors_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from rascaline import SoapPowerSpectrum

In [ ]:
HYPER_PARAMETERS = {
    "cutoff": 4.0,
    "max_radial": 4,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width": 0.5},
    },
    # "normalize": True,
}

calculator = SoapPowerSpectrum(**HYPER_PARAMETERS)

In [ ]:
from ase import io
from tqdm import tqdm

frames = io.read("/data1/simulations/datasets/rotors/simulations_cluster/results_au684/2023-10-28_01%3A23%3A35.643414/md_trajectory.traj", index="22")
frames.wrap(eps=1e-8)

In [ ]:
%%time
descriptor = calculator.compute(tqdm([frames]), gradients=["positions"])
print("before: ", len(descriptor.keys))

descriptor = descriptor.keys_to_samples("species_center")
descriptor = descriptor.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])
print("after: ", len(descriptor.keys))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


for i in range(684):
        plt.plot(descriptor.block().values[i])

plt.show()
# np.sum( descriptor.block().values[0]**2 )

for i in range(684):
        plt.plot(soap_array[i])

plt.show()

In [ ]:
descriptor.block().values[0]

# With rascal

In [ ]:
from rascal.representations import SphericalInvariants

soap_params = dict(soap_type="PowerSpectrum",
        interaction_cutoff=4.0,
        max_radial=4,
        max_angular=4,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True, # for energies gradients are ignored
        expansion_by_species_method='structure wise'
        )

hypers =  soap_params#HYPER_PARAMETERS

soap = SphericalInvariants(**hypers)

In [ ]:
%%time
managers = soap.transform(frames)
soap_array = managers.get_features(soap)

soap_grad_array = managers.get_features_gradient(soap)

In [ ]:
np.sum( soap_array[40]**2 )